In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import os
import datetime
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb
import matplotlib.patches as mpatches

In [2]:
%matplotlib inline
rcParams['figure.figsize'] = 12,6
sb.set_style('whitegrid')

In [3]:
df=pd.read_csv("Trans_data.csv")

In [14]:
df.head()

,sample_number,result_number,text_id,sampled_date,recd_date,date_completed,date_reviewed,sample_status,sample_in_spec,group_name,...,order_number,rule_type,spec_rule,min_value,max_value,text_value,sap_functional_loc,analysis_type,sap_func_loc,sap_tag_no
0,10735333,55988327,OIL/NGP-DHC-SUB-55ETDC055001/011117@1159,11/28/2016 0:00,11/15/2017 12:49,11/17/2017 18:44,11/17/2017 18:44,A,F,ELE_RELIABILITY,...,7,T,NaN,NaN,NaN,NaN,NGP-DHC-SUB-55ETDC055001,1000672303,NaN,NaN
1,10735333,55988328,OIL/NGP-DHC-SUB-55ETDC055001/011117@1159,11/28/2016 0:00,11/15/2017 12:49,11/17/2017 18:44,11/17/2017 18:44,A,F,ELE_RELIABILITY,...,8,T,NaN,NaN,NaN,NaN,NGP-DHC-SUB-55ETDC055001,1000672303,NaN,NaN
2,10735333,55988329,OIL/NGP-DHC-SUB-55ETDC055001/011117@1159,11/28/2016 0:00,11/15/2017 12:49,11/17/2017 18:44,11/17/2017 18:44,A,F,ELE_RELIABILITY,...,9,T,NaN,NaN,NaN,NaN,NGP-DHC-SUB-55ETDC055001,1000672303,NaN,NaN
3,10735333,55988330,OIL/NGP-DHC-SUB-55ETDC055001/011117@1159,11/28/2016 0:00,11/15/2017 12:49,11/17/2017 18:44,11/17/2017 18:44,A,F,ELE_RELIABILITY,...,10,T,Equal to,NaN,NaN,"Clear, free from sediment and suspended matter",NGP-DHC-SUB-55ETDC055001,1000672303,NaN,NaN
4,10735333,55988331,OIL/NGP-DHC-SUB-55ETDC055001/011117@1159,11/28/2016 0:00,11/15/2017 12:49,11/17/2017 18:44,11/17/2017 18:44,A,F,ELE_RELIABILITY,...,11,N,Result >= MIN,40.0,NaN,NaN,NGP-DHC-SUB-55ETDC055001,1000672303,NaN,NaN


In [5]:
ProbDataAll=pd.DataFrame(columns=['PD','T1','T2','T3','D1','D2','DT'])

In [6]:
TransformerList=pd.Series(df.sampling_point.unique())

In [15]:
TransformerList.head()

0    55ETDC055001_NMD
1       50XD00001_NMD
2       50XD00002_NMD
3       50XD00003_NMD
4       50XD00004_NMD
dtype: object

In [9]:
for t in range(len(TransformerList)):
    SelectTransformer=df[df.sampling_point==TransformerList[t]]
    #Sample_Number=df.SAMPLE_NUMBER.unique() 
    #Tempt_df_Single_Sample=df[df.SAMPLE_NUMBER==Sample_Number[1]]
    Tdf_Transformer_Gas=SelectTransformer[(SelectTransformer.name=='Methane - CH4')|(SelectTransformer.name=='Acetylene- C2H2')|(SelectTransformer.name=='Ethylene - C2H4')]
    Tdf_Transformer_Gas['SampleNoGas']=Tdf_Transformer_Gas['name']+' '+Tdf_Transformer_Gas['sample_number'].map(str)
    Tdf_Transformer_Gas_uniq=Tdf_Transformer_Gas.drop_duplicates(['SampleNoGas'])
    #Tdf_Transformer_Gas.to_csv('test.csv')
    Tdf_Transformer_Data=Tdf_Transformer_Gas_uniq[['sample_number','sampling_point','name','formatted_entry']].copy()
    Tdf_Transformer_Data.columns=["SampleNumber","TransformerName","Gas","PPM"]
    ###Single Transformer Data Frame manupulation
    #####for DGA_v2
    DataState=pd.DataFrame(columns=['Methane',
                          'Ethylene',
                          'Acetylene',
                          'Sum',
                          'MethanePercent',
                          'EhtylenePercent',
                          'AcetelynePercent',
                          'MethaneState',
                          'EhtyleneState',
                          'AcetelyneState',
                                   ])
    SampleList=Tdf_Transformer_Gas.sample_number.unique()
    for s in range(len(SampleList)):
        SampleData=Tdf_Transformer_Data[Tdf_Transformer_Data.SampleNumber==SampleList[s]]
        TSampleDataValues=pd.DataFrame({
            'Methane':[float(SampleData[SampleData.Gas=='Methane - CH4'].PPM)],
            'Ethylene':[float(SampleData[SampleData.Gas=='Ethylene - C2H4'].PPM)],
            'Acetylene':[float(SampleData[SampleData.Gas=='Acetylene- C2H2'].PPM)]
        })
        TSampleDataValues['Sum']=TSampleDataValues.Methane+TSampleDataValues.Ethylene+TSampleDataValues.Acetylene
        TSampleDataValues['MethanePercent']=(TSampleDataValues.Methane/TSampleDataValues.Sum)*100
        TSampleDataValues['EhtylenePercent']=(TSampleDataValues.Ethylene/TSampleDataValues.Sum)*100
        TSampleDataValues['AcetelynePercent']=(TSampleDataValues.Acetylene/TSampleDataValues.Sum)*100
        DataPercentage=TSampleDataValues
        if(float(DataPercentage.MethanePercent)<98):
            DataPercentage['MethaneState'] = 0
        else:
            DataPercentage['MethaneState'] = 1
        if(float(DataPercentage.EhtylenePercent) < 20):
              DataPercentage['EhtyleneState'] = 0
        elif(float(DataPercentage.EhtylenePercent >= 20) and float(DataPercentage.EhtylenePercent) < 23):
              DataPercentage['EhtyleneState'] = 1
        elif(float(DataPercentage.EhtylenePercent >= 23) and float(DataPercentage.EhtylenePercent) < 40):
              DataPercentage['EhtyleneState'] = 2
        elif(float(DataPercentage.EhtylenePercent >= 40) and float(DataPercentage.EhtylenePercent) < 50):
              DataPercentage['EhtyleneState'] = 3
        else:
               DataPercentage['EhtyleneState'] = 4
        if(float(DataPercentage.AcetelynePercent) < 4):
              DataPercentage['AcetelyneState'] = 0
        elif((float(DataPercentage.AcetelynePercent) >= 4) and float(DataPercentage.AcetelynePercent) < 13):
              DataPercentage['AcetelyneState'] = 1
        elif(float(DataPercentage.AcetelynePercent) >= 13 and float(DataPercentage.AcetelynePercent) < 15):
              DataPercentage['AcetelyneState'] = 2
        elif(float(DataPercentage.AcetelynePercent) >= 15 and float(DataPercentage.AcetelynePercent) < 29):
              DataPercentage['AcetelyneState'] = 3
        else: 
              DataPercentage['AcetelyneState'] = 4
        DataState=pd.concat([DataState,DataPercentage],sort=False)
    #Probability table
    ProbMethaneState0=round(((float(DataState[DataState.MethaneState==0].MethaneState.count()))/(float(DataState.shape[0]))),2)
    ProbMethaneState1=round(((float(DataState[DataState.MethaneState==1].MethaneState.count()))/(float(DataState.shape[0]))),2)
    ProbEhtyleneState0=round(((float(DataState[DataState.EhtyleneState==0].EhtyleneState.count()))/(float(DataState.shape[0]))),2)
    ProbEhtyleneState1=round(((float(DataState[DataState.EhtyleneState==1].EhtyleneState.count()))/(float(DataState.shape[0]))),2)
    ProbEhtyleneState2=round(((float(DataState[DataState.EhtyleneState==2].EhtyleneState.count()))/(float(DataState.shape[0]))),2)
    ProbEhtyleneState3=round(((float(DataState[DataState.EhtyleneState==3].EhtyleneState.count()))/(float(DataState.shape[0]))),2)
    ProbEhtyleneState4=round(((float(DataState[DataState.EhtyleneState==4].EhtyleneState.count()))/(float(DataState.shape[0]))),2)
    ProbAcetelyneState0=round(((float(DataState[DataState.AcetelyneState==0].AcetelyneState.count()))/(float(DataState.shape[0]))),2)
    ProbAcetelyneState1=round(((float(DataState[DataState.AcetelyneState==1].AcetelyneState.count()))/(float(DataState.shape[0]))),2)
    ProbAcetelyneState2=round(((float(DataState[DataState.AcetelyneState==2].AcetelyneState.count()))/(float(DataState.shape[0]))),2)
    ProbAcetelyneState3=round(((float(DataState[DataState.AcetelyneState==3].AcetelyneState.count()))/(float(DataState.shape[0]))),2)
    ProbAcetelyneState4=round(((float(DataState[DataState.AcetelyneState==4].AcetelyneState.count()))/(float(DataState.shape[0]))),2)
    ProbDataState = pd.DataFrame({'ProbMethaneState0':[ProbMethaneState0],
                                  'ProbMethaneState1':[ProbMethaneState1],
                                  'ProbEhtyleneState0':[ProbEhtyleneState0],
                                  'ProbEhtyleneState1':[ProbEhtyleneState1],
                                  'ProbEhtyleneState2':[ProbEhtyleneState2],
                                  'ProbEhtyleneState3':[ProbEhtyleneState3],
                                  'ProbEhtyleneState4':[ProbEhtyleneState4],
                                  'ProbAcetelyneState0':[ProbAcetelyneState0],
                                  'ProbAcetelyneState1':[ProbAcetelyneState1],
                                  'ProbAcetelyneState2':[ProbAcetelyneState2],
                                  'ProbAcetelyneState3':[ProbAcetelyneState3],
                                  'ProbAcetelyneState4':[ProbAcetelyneState4]})
    PD = ProbMethaneState1
    T1 = ProbMethaneState0*ProbAcetelyneState0*ProbEhtyleneState0
    T2 = (ProbMethaneState0*ProbAcetelyneState0*ProbEhtyleneState1)+(ProbMethaneState0*ProbAcetelyneState0*ProbEhtyleneState2)+(ProbMethaneState0*ProbAcetelyneState0*ProbEhtyleneState3)
    T3 = (ProbMethaneState0*ProbAcetelyneState0*ProbEhtyleneState4)+(ProbMethaneState0*ProbAcetelyneState1*ProbEhtyleneState4)+(ProbMethaneState0*ProbAcetelyneState2*ProbEhtyleneState4)
    D1 = (ProbMethaneState0*ProbAcetelyneState2*ProbEhtyleneState0)+(ProbMethaneState0*ProbAcetelyneState2*ProbEhtyleneState1)+(ProbMethaneState0*ProbAcetelyneState3*ProbEhtyleneState0)+(ProbMethaneState0*ProbAcetelyneState3*ProbEhtyleneState1)+(ProbMethaneState0*ProbAcetelyneState4*ProbEhtyleneState0)+(ProbMethaneState0*ProbAcetelyneState4*ProbEhtyleneState1)
    D2 = (ProbMethaneState0*ProbAcetelyneState2*ProbEhtyleneState2)+(ProbMethaneState0*ProbAcetelyneState3*ProbEhtyleneState2)+(ProbMethaneState0*ProbAcetelyneState4*ProbEhtyleneState2)+(ProbMethaneState0*ProbAcetelyneState4*ProbEhtyleneState3)+(ProbMethaneState0*ProbAcetelyneState4*ProbEhtyleneState4)
    DT = (ProbMethaneState0*ProbAcetelyneState1*ProbEhtyleneState0)+(ProbMethaneState0*ProbAcetelyneState1*ProbEhtyleneState1)+(ProbMethaneState0*ProbAcetelyneState1*ProbEhtyleneState2)+(ProbMethaneState0*ProbAcetelyneState1*ProbEhtyleneState3)+(ProbMethaneState0*ProbAcetelyneState2*ProbEhtyleneState3)+(ProbMethaneState0*ProbAcetelyneState3*ProbEhtyleneState3)+(ProbMethaneState0*ProbAcetelyneState3*ProbEhtyleneState4)
    ProbData=pd.DataFrame({
    'PD':[PD],
    'T1':[T1],
    'T2':[T2],
    'T3':[T3],
    'D1':[D1],
    'D2':[D2],
    'DT':[DT]
    })*100
    ProbData['Transformer'] = TransformerList[t]
    ProbData=ProbData[['Transformer','PD','T1','T2','T3','D1','D2','DT']]
    ProbDataAll=pd.concat([ProbData,ProbDataAll],sort=False)   

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
ProbDataAll

,Transformer,PD,T1,T2,T3,D1,D2,DT
0,70SYT2220KV_NMD,0.0,0.0000,0.0000,0.0000,33.0000,67.0000,0.0000
0,70SYT1220KV_NMD,0.0,0.0000,0.0000,0.0000,0.0000,100.0000,0.0000
0,72XE02_NMD,33.0,0.0000,0.0000,44.8900,0.0000,22.1100,0.0000
0,72XE01_NMD,0.0,0.0000,0.0000,100.0000,0.0000,0.0000,0.0000
0,72XD05_NMD,25.0,18.7500,0.0000,56.2500,0.0000,0.0000,0.0000
0,72XD04_NMD,0.0,0.0000,0.0000,100.0000,0.0000,0.0000,0.0000
0,72XD03_NMD,0.0,0.0000,0.0000,100.0000,0.0000,0.0000,0.0000
0,72XD02_NMD,0.0,0.0000,0.0000,100.0000,0.0000,0.0000,0.0000
0,72XD01_NMD,0.0,0.0000,0.0000,100.0000,0.0000,0.0000,0.0000
0,90TR04_NMD,0.0,0.0000,0.0000,100.0000,0.0000,0.0000,0.0000


In [12]:
ProbDataAll.to_csv('DuvalPredictData.csv')

In [13]:
PredictData = pd.read_csv("DuvalPredictData.csv")

In [ ]:
fig = plt.figure()
ax = fig.add_axes([.1, .1, 1, 1])
ax.plot(PredictData,kind='bar')
#ax.set_xticks(range(7))
ax.set_xticklabels(PredictData.Transformer, rotation=30, fontsize='large')
ax.set_title('Predict',fontsize='xx-large')
ax.set_xlabel('Gases',fontsize='x-large')
ax.set_ylabel(('Concentration (PPM)'),fontsize='x-large')